In [55]:
import random
import math

#this function is called by modinverse function to find multiplicative inverse of a where b is modulus
def gcdExtended(a, b):
    if a == 0:
        return b, 0, 1

    gcd, x1, y1 = gcdExtended(b % a, a)
    x = y1 - (b // a) * x1
    y = x1

    return gcd, x, y

#this function checks the gcd of two numbers n1 and n2
def gcd(n1, n2):
    while n2:
        n1, n2 = n2, n1 % n2
    return n1

#this function checks if a given number is a prime number
def isPrime(n):
    for i in range(2, math.isqrt(n) + 1):
        if n % i == 0:
            return False
    return True

#this function generates a random prime number of 16 bits which will be the p and q
def generate_random_prime():
    random_prime = random.randint(32768, 65535)
    while not isPrime(random_prime):
        random_prime = random.randint(32768, 65535)
    return random_prime

#this function generates value of public key modulus N
def evaluate_N(p, q):
    return p * q

#this function generates value of phi_n
def evaluate_phi_n(p, q):
    return (p - 1) * (q - 1)

#this function generates the value of public key exponent e
def evaluate_e(phi_n):
    e = random.randint(2, phi_n // 65536)
    while gcd(phi_n, e) != 1 or e >= phi_n:
        e = random.randint(2, phi_n // 65536)
    return e

#this function generates the value of decryption private key d
def modInverse(e, phi_n):
    gcd, x, y = gcdExtended(e, phi_n)
    if gcd != 1:
        raise ValueError("Inverse doesn't exist")

    return (x % phi_n + phi_n) % phi_n

#this function calculates the value of a ^ b % mod using square and multiply method
def square_multiply(a, b, mod):
    result = 1
    a = a % mod

    while b > 0:
        if b % 2 == 1:
            result = (result * a) % mod
        b //= 2
        a = (a * a) % mod

    return result

#this function encrypts a given message using partner's public key e and N
def encrypt_message(input_string, e, N):
    m_chunks = []
    int_chunks = []
    e_chunks = []

    for i in range(0, len(input_string), 3):
        chunk = input_string[i:i + 3]
        
        m_chunks.append(chunk)

        hex_chunk = ''.join([format(ord(c), '02x') for c in chunk])

        int_chunks.extend([int(hex_chunk[j:j + 16], 16) for j in range(0, len(hex_chunk), 16)])

    for msg in int_chunks:
        encrypted_chunk = square_multiply(msg, e, N)
        e_chunks.append(encrypted_chunk)
    
    print("My message Chunks:", ','.join(map(str, m_chunks)))
        
    return e_chunks

#this function decrypts the encrypted message received from partner using private key d
def decrypt_message(encrypted_chunks, d, N):
    d_chunks = []
    decrypted_string = ""

    for encrypted_chunk in encrypted_chunks:
        decrypted_chunk = square_multiply(encrypted_chunk, d, N)

        decrypted_hex_chunk = format(decrypted_chunk, 'x')

        for k in range(0, len(decrypted_hex_chunk), 2):
            byte_string = decrypted_hex_chunk[k:k + 2]
            byte = chr(int(byte_string, 16))
            decrypted_string += byte
            
    for i in range(0, len(decrypted_string), 3):
        chunk = decrypted_string[i:i + 3]
        d_chunks.append(chunk)
            
    print("Partner's message Chunks:", ','.join(map(str, d_chunks)))

    return decrypted_string

#this function signs my message to be verified using my private key d and my N
def sign_message(input_string, d, N):
    s_chunks = []
    int_chunks = []
    e_chunks = []

    for i in range(0, len(input_string), 3):
        chunk = input_string[i:i + 3]
        
        s_chunks.append(chunk)

        hex_chunk = ''.join([format(ord(c), '02x') for c in chunk])

        int_chunks.extend([int(hex_chunk[j:j + 16], 16) for j in range(0, len(hex_chunk), 16)])

    for msg in int_chunks:
        encrypted_chunk = square_multiply(msg, d, N)
        e_chunks.append(encrypted_chunk)
    
    print("My Signed message Chunks:", ','.join(map(str, s_chunks)))
        
    return e_chunks

#this function verifies the partner's signed message using his/her public e and N 
def verify_received_signed_msg(encrypted_chunks, e, N, partner_signed_msg):
    decrypted_string = ""

    for encrypted_chunk in encrypted_chunks:
        decrypted_chunk = square_multiply(encrypted_chunk, e, N)

        decrypted_hex_chunk = format(decrypted_chunk, 'x')

        for k in range(0, len(decrypted_hex_chunk), 2):
            byte_string = decrypted_hex_chunk[k:k + 2]
            byte = chr(int(byte_string, 16))
            decrypted_string += byte
            
    if partner_signed_msg == decrypted_string:
        return True
    else:
        return False

def main():
    random.seed()

#     p = generate_random_prime()
#     q = generate_random_prime()
    p = 62189;
    q = 52859;
    N = evaluate_N(p, q)
    phi_n = evaluate_phi_n(p, q)
#     e = evaluate_e(phi_n)
    e = 3817
    d = modInverse(e, phi_n)
    
    partner_n = 3346256237
    partner_e = 32117
    
    print("My paramters are:\n")
    
    print(f"P = {p}\nQ = {q}\nN = {N}\nphi_n = {phi_n}\ne = {e}\nd = {d}\n")
    
    print("My partner's public key components are:\n")
    
    print(f"partner_n = {partner_n}\npartner_e = {partner_e}\n")
    
    print("Encryption of My Message:\n")
    
    input_string = "Aim for the impossible"

    print("My message to encrypt:", input_string)

    e_cipher_chunks = encrypt_message(input_string, partner_e, partner_n)

    print("My ciphertext to send:", ', '.join(map(str, e_cipher_chunks)))
    
    print("\nDecryption of Partner's Message:")

    received_cipher_chunks = [3154750071, 2795536361, 2173849509, 565069380, 943468783, 1457791096, 1636785221, 446176987, 219107125]

    print("\nPartner's ciphertext received:", ', '.join(map(str, received_cipher_chunks)))

    decrypted_message = decrypt_message(received_cipher_chunks, d, N)

    print("Partner's decrypted message:", decrypted_message)
    
    print("\nSign My Message:")
    
    my_signed_message = "Aniket Agarwal"
    
    print("\nMy  message to be signed:", my_signed_message)
    
    my_s_chunks = sign_message(my_signed_message, d, N)
    
    print("My signed ciphertext to send:", ', '.join(map(str, my_s_chunks)))
    
    print("\nVerify Partner's Sign:")
    
    partner_signed_msg = "Anab Abdi"
    
    partner_signed_ciphertext = [1943182935, 2888579549, 2241914581]
    
    print(f"\nPartner's signed message = {partner_signed_msg}\nPartner's signed ciphertext = {partner_signed_ciphertext}")
    
    partner_verified_signed_msg =  verify_received_signed_msg(partner_signed_ciphertext, partner_e, partner_n, partner_signed_msg)
    
    print("Is partner's signed message a valid signature:", partner_verified_signed_msg)
    
    

if __name__ == "__main__":
    main()



My paramters are:

P = 62189
Q = 52859
N = 3287248351
phi_n = 3287133304
e = 3817
d = 2251992033

My partner's public key components are:

partner_n = 3346256237
partner_e = 32117

Encryption of My Message:

My message to encrypt: Aim for the impossible
My message Chunks: Aim, fo,r t,he ,imp,oss,ibl,e
My ciphertext to send: 30966275, 1791799262, 1254862435, 1921378134, 2989293761, 2712515199, 1120956057, 2170497989

Decryption of Partner's Message:

Partner's ciphertext received: 3154750071, 2795536361, 2173849509, 565069380, 943468783, 1457791096, 1636785221, 446176987, 219107125
Partner's message Chunks: RSA, by, Ri,ves,tSh,ami,rAd,lem,an
Partner's decrypted message: RSA by RivestShamirAdleman

Sign My Message:

My  message to be signed: Aniket Agarwal
My Signed message Chunks: Ani,ket, Ag,arw,al
My signed ciphertext to send: 2583198294, 1844306159, 69547231, 623892297, 248110039

Verify Partner's Sign:

Partner's signed message = Anab Abdi
Partner's signed ciphertext = [1943182935, 